# Make YOLO CustomDataSet

### 전체 데이터셋에서 필요한 데이터 추출 및 어노테이션 수정
- 활용 데이터 셋 : hub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=153
- 어노테이션 형식 : json -> txt(Yolo)

In [22]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### 데이터의 작물 부위 확인
- 사용하는 데이터셋에선 줄기 및 열매, 잎 등 여러가지 식물 부위의 데이터가 있다. 우리는 잎 감지를 위해 데이터를 사용하기 때문에 잎 사진 데이터를 확인해야된다.
- 예시)
  - 이미지 파일명: V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg
  - 어노테이션 파일명: V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg.json
    - 구분자 '_'로 토큰을 나누면 각각의 토큰에 의미를 부여할 수 있다. 이는 데이터 셋 생성자가 정해놓은 규칙으로 작성되어 있는데 중요한 토큰들만 살펴보자
      - token[1] : 세부과제
        - '77': 시설작물 질병, '78':노지작물 해충, '79' :노지작물 질병, '80': 과수화상병
      - token[2] : 데이터 종류
        - '0' : 정상 작물, '1': 질병, '2': 해충, '3':충해
      - token[4] : 작물 코드
        - 별첨 1
      - token[5] : 작물의 부위별 코드
        - 별첨 2

#### 별첨
##### 별첨 1 (식물 종류)
00 작물 없음
01 가지
02 고추
03 단호박
04 딸기
05 상추
06 수박
07 애호박
08 오이
09 쥬키니호박
10 참외
11 토마토
12 포도
##### 별첨 2 (식물 부위)
00 구분 없음
01 열매
02 꽃
03 잎
04 가지
05 줄기
06 뿌리
07 해충

In [23]:
import re

def is_area(filename,area_code):
    # '_'를 기준으로 분할하고 6번째 요소(식물 부위 코드) 확인 -> 해당 요소가 객체의 위치를 나타난다.
    token = filename.split('_')
    #print(parts)
    if len(token) >= 7:
        if token[5] == area_code: # 03번은 잎이다.
            return True
        else:
            return False
    else:
        return False

### 데이터 어노테이션 형식 변환
- 기존 데이터셋의 어노테이션은 json형식의 커스텀 어노테이션 형식을 고수한다. 우리는 YOLO모델을 사용하여 학습시키기때문에 YOLO 어노테이션(txt)형식으로 변환해준다.
  - Yolo 어노테이션 형식
    - ex) class_code x_center y_center box_width box_height

In [24]:
def convert_to_yolo_annotation(path):
    # JSON 파일 읽기
    with open(path, 'r') as f:
        json_data = json.load(f)
        json_string = json.dumps(json_data)
        data = json.loads(json_string)
        # 이미지 너비와 높이
        image_width = data["description"]["width"]
        image_height = data["description"]["height"]

        # 바운딩 박스 좌표
        points = data["annotations"]["points"][0]

    # center구하고 데이터 정규화
    x_center = (points["xtl"] + points["xbr"]) / 2 / image_width
    y_center = (points["ytl"] + points["ybr"]) / 2 / image_height
    box_width = (points["xbr"] - points["xtl"]) / image_width
    box_height = (points["ybr"] - points["ytl"]) / image_height

    # YOLO 어노테이션 형식 리스트 반환
    return [x_center, y_center, box_width, box_height]

### Custom Dataset 폴더 구조 생성
- Yolo train 에서 지정한 데이터 폴더 구조를 생성한다.
- Yolo train 폴더 구조
  - CustomDataset <br>
  ㄴ train/<br>
  &nbsp;&nbsp;&nbsp; 1.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 1.txt(어노테이션 파일)<br>
  ㄴ valid/<br>
  &nbsp;&nbsp;&nbsp; 2.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 2.txt(어노테이션 파일)<br>
  ㄴ test/<br>
  &nbsp;&nbsp;&nbsp; 3.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 3.txt(어노테이션 파일)<br>
   .yaml<br>

In [25]:
import os
import shutil

def set_Folder(name, path):
    # train,valid,test 파일의 경로 저장한 리스트
    result = []
    Project_path = os.path.join(path, name)

    if os.path.exists(Project_path):
      delete_folder(Project_path)

    os.makedirs(Project_path)
    print(f"Destination folder '{Project_path}' created.")

    for folder_name in ["train", "valid", "test"]:
      new_folder_path = os.path.join(Project_path, folder_name)
      result.append(new_folder_path)

      os.makedirs(new_folder_path)
      print(f"Destination folder '{new_folder_path}' created.")

    return result

def delete_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Folder '{path}' deleted successfully.")
    except OSError as e:
        print(f"Error: {path} : {e.strerror}")

### 파일 저장
- 생성한 어노테이션 정보를 .txt파일의 형태로 저장한다.
- 조건에 만족하는 이미지 파일을 특정 폴더에 복사한다.
  - store_annotation(output_path,basename,data): 생성한 yolo 데이터 어노테이션 형식을 실제 파일로 저장한다.
    - output_path 에 data를 basename으로 저장한다
  - store_images(source_path, output_path): 이미지를 특정 폴더에 복사하여 저장한다.
    - source_path에 있는 이미지파일을 output_path경로에 저장한다.
  - store_all(images, labels, folder): 이미지와 어노테이션 파일을 특정 폴더에 저장한다.
    - images_list와 labels_list에 있는 데이터를 folder에 파일로 저장한다.

In [26]:
import json

def store_annotation(output_folder,basename,data):
    # 최종 어노테이션 문자열을 저장하는 list
    result = []

    bbox_string = " ".join([str(x) for x in data])
    result.append(f"0 {bbox_string}")
    if result:
      # 확장자 제거
      basename = basename.split(".")[0]
      with open(os.path.join(output_folder, f"{basename}.txt"), "w", encoding="utf-8") as f:
          f.write("\n".join(result))
      return True
    return False


In [27]:
# 학습에 활용할 이미지 데이터를 목적지 폴더(학습 폴더 구조)에 맞춰 복사하는 함수
def store_images(source_path, output_path):

  filename = os.path.basename(source_path)

  destination_path = os.path.join(output_path, filename)

  try:
      # Copy the file
      shutil.copy2(source_path, destination_path)
  except Exception as e:
      print(f"Error occurred while copying the file: {e}")


In [28]:
def store_all(images, labels, folder):
  labels_cnt = 0
  for i in images:
      store_images(i,folder)

  images_cnt = sum([len(files) for _, _, files in os.walk(folder)])
  for i in labels:
    basename= os.path.basename(i)
    if store_annotation(folder,basename,convert_to_yolo_annotation(i)):
      labels_cnt += 1

  print(labels_cnt, images_cnt)

### 실행

##### 환경변수 설정

- origin_labels_path: 기존 데이터셋의 라벨링 파일이 위치한 폴더 경로입니다.
- origin_images_path: 기존 데이터셋의 이미지 파일이 위치한 폴더 경로입니다.
- Dataset_name: 새로 생성할 커스텀 데이터셋의 이름을 설정합니다.
- Custom_dataset_path: 커스텀 데이터셋을 저장할 폴더의 경로를 지정합니다.
- extension_label: 라벨 데이터가 가지고 있는 확장자를 모아둔 리스트입니다.
- extension_image: 이미지 데이터가 가지고 있는 확장자를 모아둔 리스트입니다.
- area_code : 어떤 식물 부위를 사용할지 지정하는 변수입니다.

In [29]:
origin_labels_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/참외/라벨링데이터/0.정상/"
origin_images_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/참외/원천데이터/0.정상/"
Dataset_name = "YOLO_CustomDataSet"
Custom_dataset_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/"
# Custom_dataset_path = "/content/"
extension_label = ["json","txt"]
extension_image = ["jpg","JPG","png","PNG"]
area_code = "03"

#### 기존 데이터셋 파일 불러오기

In [30]:
from glob import glob

all_labels = []
all_images = []

# 특정 확장자를 가진 파일 읽어오기
for ext_label in extension_label:
  all_labels.extend(glob(f'{origin_labels_path}*.{ext_label}'))
for ext_image in extension_image:
  all_images.extend(glob(f'{origin_images_path}*.{ext_image}'))

labels_path_list = list()
images_path_list = list()

# 식물 부위 코드가 일치하는 이미지 및 라벨링 데이터만 선별
for i in all_labels:
    basename= os.path.basename(i)
    if is_area(basename,area_code):
      labels_path_list.append(i)

for i in all_images:
    basename= os.path.basename(i)
    if is_area(basename,area_code):
      images_path_list.append(i)


#### 실제 커스텀 데이터셋 폴더에 저장

In [31]:
# 훈련 데이터 90%, 검증데이터 10%
split_index1 = int(len(images_path_list) * 0.95)
split_index2 = int(len(images_path_list) * 0.98)
train_images = images_path_list[:split_index1]
train_labels = labels_path_list[:split_index1]
valid_images = images_path_list[split_index1:split_index2]
valid_labels = labels_path_list[split_index1:split_index2]
test_images = images_path_list[split_index2:]
test_labels = labels_path_list[split_index2:]

# 커스텀 데이터셋을 관리할 폴더 구조를 생성한다.
folder_list = set_Folder(Dataset_name, Custom_dataset_path)

# folder_list = [train경로, valid경로, test경로]
for index, folder in enumerate(folder_list):
  if index == 0: # train
    store_all(train_images,train_labels,folder)
  if index == 1: # vaild
    store_all(valid_images,valid_labels,folder)
  if index == 2: # test
    store_all(test_images,test_labels,folder)

Folder '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet' deleted successfully.
Destination folder '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet' created.
Destination folder '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/train' created.
Destination folder '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/valid' created.
Destination folder '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/test' created.
273 273
9 9
6 6
